In [26]:
import random
import re
import time
from datetime import datetime, timedelta
from functools import partial

import numpy as np
import pandas as pd
import torch
from sklearn.utils import shuffle
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, random_split, RandomSampler
from torch.optim.adamw import AdamW
from transformers import BertForSequenceClassification, BertConfig, AutoTokenizer, AutoModel
from transformers import get_linear_schedule_with_warmup

from ml.src.data.load_datasets import load_RuTweetCorp, DATA_PATH
from ml.src.models.rubert import BertClassifier

In [27]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
%autoreload 2

In [29]:
# data_path = load_RuTweetCorp(concat=True, overwrite=True)

In [30]:
data_path = DATA_PATH / 'raw' / 'ru-tweet-corp.csv'

In [31]:
data = pd.read_csv(data_path, header=None)

In [32]:
# data = shuffle(data)

In [33]:
data.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
5,5,408906761769598976,1386325943,JustinB94262583,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1,0,0,0,40,6,16,0
6,6,408906762436481024,1386325943,twinkleAYO,"RT @SpoonLamer: Ох,900 :D ну это конечно же @t...",1,0,1,0,5169,58,43,2
7,7,408906764114206720,1386325944,pycalyruhog,RT @veregijytaqo: У тебя есть ухажёр? Нет - мо...,1,0,2,0,393,112,153,0
8,8,408906764608749568,1386325944,grishintv,Поприветствуем моего нового читателя @Alexey17...,1,0,0,0,5872,1387,1431,12
9,9,408906765841870848,1386325944,alina_612,Теперь у меня есть частичка Сиднея :) #Sydney ...,1,0,0,0,263,5,36,0


In [34]:
X, y = data.iloc[:, 4], data.iloc[:, 5]

In [35]:
X.dropna(inplace=True)
y.dropna(inplace=True)

In [36]:
y = y.replace(-1, 0)

In [37]:
X.head(10)

0    @first_timee хоть я и школота, но поверь, у на...
1    Да, все-таки он немного похож на него. Но мой ...
2    RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3    RT @digger2912: "Кто то в углу сидит и погибае...
4    @irina_dyshkant Вот что значит страшилка :D\nН...
5    ну любишь или нет? — Я не знаю кто ты бля:D ht...
6    RT @SpoonLamer: Ох,900 :D ну это конечно же @t...
7    RT @veregijytaqo: У тебя есть ухажёр? Нет - мо...
8    Поприветствуем моего нового читателя @Alexey17...
9    Теперь у меня есть частичка Сиднея :) #Sydney ...
Name: 4, dtype: object

In [38]:
y.head(10)

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: 5, dtype: int64

In [39]:
y.value_counts()

1    114911
0    111923
Name: 5, dtype: int64

In [46]:
model = BertClassifier('DeepPavlov/rubert-base-cased', 'DeepPavlov/rubert-base-cased', epochs=1, model_save_path='../../models/rubert.pt')

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [47]:
model.preparation(X[:12_000], y[:12_000])

D:\projects\nyaural_nyatworks\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [48]:
model.train()

Epoch 1/1


 18%|██████████▌                                                 | 53/300 [20:52<1:37:15, 23.62s/it]


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 9830400 bytes.